## Custom vision 

### Inference example using tensorflow


#### Initial Setup

Once we have exported our tensorflow model from Custom vision portal, we are ready to load it using the tensorflow framework

In [ ]:
import tensorflow as tf
import numpy as np
import os

from PIL import Image

print(tf.version)

filename = 'models/model.pb'
labels_filename = 'models/labels.txt'

graph_def = tf.compat.v1.GraphDef()
labels = []

# Import the TF graph
with tf.compat.v1.gfile.GFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

# Create a list of labels.
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())

#### Inference script

Once we have loaded the tensorflow graph of the model, we can use one of our tests images to develop our inference script. 

- Resize our image to fit the input size of our model.
- Transpose the tensor from RGB color order to BGR.
- Indentify the input node and the output layer from our graph. We can use the netron application to see our graph.
- Obtain the max probability of our classification class


In [ ]:
# Load from a file
imageFile = "images/test/test_image.jpg"
image = Image.open(imageFile)

# Resize 
image = image.resize((224, 224), resample=Image.BILINEAR)

# Convert to numpy array - tensor
image_tensor = np.asarray(image)

# Convert RGB -> BGR
r,g,b = np.array(image_tensor).T
image_tensor = np.array([b,g,r]).transpose()

print("Numpy array mode=BGR shape={}".format(image_tensor.shape))

# These names are part of the model and cannot be changed.
output_layer = 'loss:0'
input_node = 'Placeholder:0'

with tf.compat.v1.Session() as sess:
    prob_tensor = sess.graph.get_tensor_by_name(output_layer)
    predictions, = sess.run(prob_tensor, {input_node: [image_tensor] })
    
print(f"Predictions: {predictions}")

# Print the highest probability label
highest_probability_index = np.argmax(predictions)
print('Classified as: ' + labels[highest_probability_index])

# Or you can print out all of the results mapping labels to probabilities.
label_index = 0
for p in predictions:
    truncated_probablity = np.float64(np.round(p,4))
    print(labels[label_index], truncated_probablity)
    label_index += 1